# 2018 Results Analysis

This notebook contains some analysis of the results of the 2018 season. It includes all Wednesday and Friday evening races over the season but excludes mayhems, pursuit races, club champs and winter series.

Start by defining some useful functions.

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


column_headings = ["HelmName", "Class", "PY", "SailNo", "Fleet", "Rank", "Elapsed", "Corrected", "Points", "Reg No.", "Reg Date"]
alternative_headings = ["HelmName", "Class", "PY", "SailNo", "Fleet", "Rank", "Place", "Points", "Reg No.", "Reg Date"]
fleets = ['Superfast', 'Fast', 'Medium', 'Slow']

plt.rcParams['figure.figsize'] = [20, 10]


def scrape_race(url):
    """Scrape all the results for a single race."""
    print(url)
    with urllib.request.urlopen(url) as response:
       html = response.read()
    
    soup = BeautifulSoup(html)
    
    tables = soup.findAll('table')
    return tables[:4]


def get_dataframe_from(table):
    """Turn a table into a data frame."""
    table_rows = table.find_all('tr')
    l = []
    for tr in table_rows[1:]:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
        
    # Different column headers are used if there were no results recorded in a fleet.
    if l and len(l[0]) == 11:        
        df = pd.DataFrame(l, columns=column_headings)
    else:
        df = pd.DataFrame(l, columns=alternative_headings)
        
    return df

Import all results for all Wednesday and Friday races.

In [ ]:
base_url = 'https://www.warsashsc.org.uk/results/18'
series = ['wa', 'wb', 'wc', 'fa', 'fb', 'fc']

all_races_frames = []
race_on = []

for s in series:
    for i in range(1, 9):
        try:
            race = scrape_race(base_url + s + str(i) + '.htm')
            
            for table in race:
                df = get_dataframe_from(table)
                df['Day'] = s[0]
                df['Series'] = s[1]
                df['Race'] = i

                all_races_frames.append(df)
                
            race_on.append(True)
        except:
            print('No racing')
            race_on.append(False)
        
        
            
all_races = pd.concat(all_races_frames, sort=True)

## Race Stats

In [ ]:
print('There were a total of %i results recorded.' % len(all_races.index))

print('There were a total of %i OCSs.' % (all_races['Elapsed'] == 'OCS').sum())

print('There were a total of %i duties recorded.' %(all_races['Elapsed'] == 'Duty').sum())

print('There were a total of %i DNFs recorded.' % (all_races['Elapsed'] == 'DNF').sum())

excluding_duties = all_races[~all_races['Elapsed'].isin(['Duty'])]
print('There were %i results posted excluding duties.' % len(excluding_duties.index))

only_finishes = all_races[~all_races['Elapsed'].isin(['Duty', 'OCS', 'DNE', 'DNF', 'DSQ', 'AVG'])]
print('There were %i finishes recorded.' % len(only_finishes.index))

## Class Analysis

In [ ]:
classes_frequency = excluding_duties['Class'].value_counts()

py_by_class = {}
for c in excluding_duties['Class'].unique():
    py_by_class[c] = excluding_duties.loc[excluding_duties['Class'] == c, 'PY'].iloc[0]

print('%i different classes raced in 2018' % len(classes_frequency))
print(classes_frequency)

In [ ]:
pys = excluding_duties['PY'].astype(int).tolist()

plt.hist(pys, bins=30)
plt.xlabel('PY')
plt.xticks(np.arange(min(pys), max(pys), 50))
plt.title('Distribution of races completed by PY')

## Series Analysis

In [ ]:
print('Wednesday series total results')
print(excluding_duties[excluding_duties['Day'] == 'w']['Series'].value_counts())
print('Friday series total results')
print(excluding_duties[excluding_duties['Day'] == 'f']['Series'].value_counts())

In [ ]:
# Count the numbers in each fleet in each Wednesday race.
fleet_numbers = {}
for fleet in fleets:
    fleet_numbers[fleet] =[]

wednesdays = excluding_duties[excluding_duties['Day'] == 'w']
for s in ['a', 'b', 'c']:
    wed_series = wednesdays[wednesdays['Series'] == s]
    for r in range(1, 9):
        race_numbers = wed_series[wed_series['Race'] == r]['Fleet'].value_counts()
        for fleet in fleets:
            if fleet in race_numbers:
                fleet_numbers[fleet].append(race_numbers[fleet])
            else:
                fleet_numbers[fleet].append(0)

In [ ]:
for fleet in fleets:
    # Remove cancelled races
    frequencies = [n for i, n in enumerate(fleet_numbers[fleet]) if race_on[i]]
                   
    print(fleet)
    print('Min = %i' % min(frequencies))
    print('Max = %i' % max(frequencies))
    print('Mean = %f' % np.mean(frequencies))
    print('Median = %i' % np.median(frequencies))
    print()

In [ ]:
for fleet in fleets:
    plt.plot((fleet_numbers[fleet]), label=fleet)
plt.legend(fleets)
plt.xlabel('Race')
plt.ylabel('Number in Fleet')
plt.title('Numbers in Each Fleet on Wednesdays')

## Race Lengths

In [ ]:
ftr = [3600, 60, 1]
i = 0
lengths_by_fleet = {}
for fleet in fleets:
    lengths_by_fleet[fleet] =[]
    
wednesday_finishes = only_finishes[only_finishes['Day'] == 'w']
for s in ['a', 'b', 'c']:
    wed_series = wednesday_finishes[wednesday_finishes['Series'] == s]
    for r in range(1, 9):
        if race_on[i]:
            results_for_race = wed_series[wed_series['Race'] == r]
            for fleet in fleets:
                results_for_fleet = results_for_race[results_for_race['Fleet'] == fleet]
                for i, result in results_for_fleet.iterrows():
                    try:
                        time_in_s = sum([a*b for a,b in zip(ftr, map(int, result['Elapsed'].split(':')))])
                        lengths_by_fleet[fleet].append(time_in_s)
                    except Exception as e:
                        pass
    i += 1

In [ ]:
for fleet in fleets:                  
    print(fleet)
    print('Min = %i' % min(lengths_by_fleet[fleet]))
    print('Max = %i' % max(lengths_by_fleet[fleet]))
    print('Mean = %f' % np.mean(lengths_by_fleet[fleet]))
    print('Median = %i' % np.median(lengths_by_fleet[fleet]))
    print()

In [ ]:
plt.hist(lengths_by_fleet['Superfast'], bins=20)
plt.title('Distribution of Wednesday Race Times for Superfast')

In [ ]:
plt.hist(lengths_by_fleet['Fast'], bins=20)
plt.title('Distribution of Wednesday Race Times for Fast')

In [ ]:
plt.hist(lengths_by_fleet['Medium'], bins=20)
plt.title('Distribution of Wednesday Race Times for Medium')

In [ ]:
plt.hist(lengths_by_fleet['Slow'], bins=20)
plt.title('Distribution of Wednesday Race Times for Slow')

## Sailors

In [ ]:
excluding_duties['HelmName'].value_counts()

## 2019 Fleet Proposals

In [ ]:
def display_fleet_proposal_stats(fast, medium, slow):
    """Display fleet stats for a given proposal."""
    fleet_numbers = {}
    for fleet in fleets:
        fleet_numbers[fleet] =[]

    wednesdays = excluding_duties[excluding_duties['Day'] == 'w']
    for s in ['a', 'b', 'c']:
        wed_series = wednesdays[wednesdays['Series'] == s]
        for r in range(1, 9):
            for fleet in fleets:
                fleet_numbers[fleet].append(0)
            for i, result in wed_series[wed_series['Race'] == r].iterrows()
                py = result['PY']
                if py > fast:
                    fleet = 'Superfast'
                elif py > medium:
                    fleet = 'Fast'
                elif py > slow:
                    fleet = 'Slow'
                fleet_numbers[fleet][-1] += 1
                    
    for fleet in fleets:
        # Remove cancelled races
        frequencies = [n for i, n in enumerate(fleet_numbers[fleet]) if race_on[i]]

        print(fleet)
        print('Min = %i' % min(frequencies))
        print('Max = %i' % max(frequencies))
        print('Mean = %f' % np.mean(frequencies))
        print('Median = %i' % np.median(frequencies))
        print()

In [ ]:
display_fleet_proposal_stats(1025, 1100, 1200)